# Black Scholes Exercise 6: MPI implementation
Use MPI to parallelize and distribute the work

In [ ]:
# Boilerplate for the example

import cProfile
import pstats
import numpy as np

try:
    import numpy.random_intel as rnd
except:
    import numpy.random as rnd

# make xrange available in python 3
try:
    xrange
except NameError:
    xrange = range

SEED = 7777777
S0L = 10.0
S0H = 50.0
XL = 10.0
XH = 50.0
TL = 1.0
TH = 2.0
RISK_FREE = 0.1
VOLATILITY = 0.2
TEST_ARRAY_LENGTH = 1024

###############################################

def gen_data(nopt):
    return (
        rnd.uniform(S0L, S0H, nopt),
        rnd.uniform(XL, XH, nopt),
        rnd.uniform(TL, TH, nopt),
        )

nopt=100000
price, strike, t = gen_data(nopt)
call = np.zeros(nopt, dtype=np.float64)
put  = -np.ones(nopt, dtype=np.float64)

In [1]:
# Let's add a Black Scholes kernel (numpy) and an MPI wrapper

from numpy import log, invsqrt, exp, erf
# Black Scholes kernel
def black_scholes(nopt, price, strike, t, rate, vol):
    mr = -rate
    sig_sig_two = vol * vol * 2

    P = price
    S = strike
    T = t

    a = log(P / S)
    b = T * mr

    z = T * sig_sig_two
    c = 0.25 * z
    y = invsqrt(z)

    w1 = (a - b + c) * y
    w2 = (a - b - c) * y

    d1 = 0.5 + 0.5 * erf(w1)
    d2 = 0.5 + 0.5 * erf(w2)

    Se = exp(b) * S

    call = P * d1 - Se * d2
    put = call - P + Se
    
    return (call, put)


# MPI wrapper
from mpi4py import MPI
def black_scholes_mpi(nopt, price, strike, t, rate, vol):
    comm = MPI.COMM_WORLD
    nump = comm.size
    noptpp = int(nopt/nump)

    myprice = np.empty(noptpp, dtype=np.float64)
    mystrike = np.empty(noptpp, dtype=np.float64)
    myt = np.empty(noptpp, dtype=np.float64)

    # Scatter data into arrays
    comm.Scatter(price, myprice, root=0)
    # TODO scatter strike and t

    mycall, myput = black_scholes(noptpp, myprice, mystrike, myt, rate, vol)

    comm.Gather(mycall, call)
    #TODO gather put

    return call, put

In [ ]:
# Running a single python function in MPI is possible with ipython but can be tricky to setup.
# Let's run the entire program with MPI.
# We need to add our own timing.
from time import clock
t1 = clock()
black_scholes_mpi(nopt, price, strike, t, RISK_FREE, VOLATILITY)
t = (clock()-t1)*1000
print("Time: {:.2f}ms".format(t))

Now we can safe the code to a file and run it with mpirun

In [ ]:
%save -f runme.py 1-3
!mpirun -n 4 python ./runme.py

## 1. Let only rank 0 print the timing
## 2. How can we reduce the communication cost?